In [3]:
import os
import sys
import re
import time
import random
import warnings
import collections
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

sys.path.append('../../src')
import cb_utils

sns.set(style="darkgrid")
pd.options.display.max_columns = 500

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Milliman MANI File ETL

### Config

In [4]:
data_dir = '/Users/bp/workspace/msh/milliman_data'
current_folder = '20241203'
# pro_file_name = 'ACO Builder MSSP Prospective Explorer v2025.3.txt'
# retro_file_name = 'ACO Builder MSSP Retrospective Explorer v2025.3.txt'
table_prefix = 'milliman_mani'

### DataTables

In [5]:
file_name = 'MANI CY2021 Data Tables - v2024.2.0 - Nationwide.csv'
year = 2021
mani_data_tables_2020_df = pd.read_csv(f'{data_dir}/{current_folder}/{file_name}',  encoding='utf-8-sig', low_memory=False)
# display(mani_data_tables_2020_df.head())
mani_data_tables_2020_df = cb_utils.df_format_columns(mani_data_tables_2020_df)
mani_data_tables_2020_df.head()

,year,aco_builder_id,tin_ccn_type,tin_ccn_id,provider_name,pac_id,st_msa,carrier,member_status,py,risk_score_mmwtd,risk_score_mmwtd_region,ma_benchmark_rate_carrier,ma_benchmark_rate_0pct,ma_bid_revenue,paid_atffsreimb,paid_atffsreimb_region,ip_admits,ip_admits_imputed,ip_admits_region,ip_days,ip_days_imputed,ip_days_region,snf_admits,snf_admits_imputed,snf_admits_region,snf_days,snf_days_imputed,snf_days_region,ed,ed_imputed,ed_region,phy_office,phy_office_imputed,phy_office_region,rad_adv_imag_op,rad_adv_imag_op_imputed,rad_adv_imag_op_region,rad_adv_imag_nonhosp,rad_adv_imag_nonhosp_imputed,rad_adv_imag_nonhosp_region,rad_gen_op,rad_gen_op_imputed,rad_gen_op_region,rad_gen_nonhosp,rad_gen_nonhosp_imputed,rad_gen_nonhosp_region,rad_therap_op,rad_therap_op_imputed,rad_therap_op_region,rad_therap_nonhosp,rad_therap_nonhosp_imputed,rad_therap_nonhosp_region,prev_physicals,prev_physicals_imputed,prev_physicals_region,surgery_asc,surgery_asc_imputed,surgery_asc_region,surgery_op,surgery_op_imputed,surgery_op_region,part_b_rx_nonhosp,part_b_rx_nonhosp_imputed,part_b_rx_nonhosp_region,part_b_rx_op,part_b_rx_op_imputed,part_b_rx_op_region,lab_nonhosp,lab_nonhosp_imputed,lab_nonhosp_region,lab_op,lab_op_imputed,lab_op_region
0,2021,00004B7C4023,TIN,474454561.0,RINDFLEISCH FAMILY PRACTICE PLLC,9335446202,ID-26820,"Blue Cross of Idaho Health Services, Inc.",NaN,56.91667,743.40115,705.62878,728695.02220,7.277346e+05,648336.78700,6.644800e+05,6.474441e+05,16.93342,16.93342,12.32244,87.30702,31.30702,75.18781,NaN,NaN,2.90598,NaN,NaN,63.99426,92.11521,6.11521,32.68220,500.61404,4.61404,423.89944,47.09132,3.09132,42.77029,17.08657,17.08657,8.55608,72.24105,4.24105,65.73006,44.17988,6.17988,52.70916,NaN,NaN,4.47132,NaN,NaN,1.39879,20.46610,7.46610,28.32931,12.0,NaN,4.61939,16.05092,16.05092,28.65054,65.14031,3.14031,52.68929,4.42054,4.42054,60.51826,383.0,NaN,603.62667,110.00000,NaN,433.64913
1,2021,00004B7C4023,TIN,474454561.0,RINDFLEISCH FAMILY PRACTICE PLLC,9335446202,ID-26820,CVS Health Corporation,NaN,2.00000,21.77081,22.23206,22266.01646,2.158005e+04,18505.30109,2.016839e+04,2.074453e+04,NaN,NaN,0.36720,NaN,NaN,0.87216,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.49360,2.87014,2.87014,10.25160,NaN,NaN,1.60656,NaN,NaN,0.22932,NaN,NaN,1.65144,NaN,NaN,0.64284,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.01004,NaN,NaN,0.22932,NaN,NaN,1.80432,NaN,NaN,1.59144,NaN,NaN,1.62144,135.0,NaN,27.31944,NaN,NaN,15.94860
2,2021,00004B7C4023,TIN,474454561.0,RINDFLEISCH FAMILY PRACTICE PLLC,9335446202,ID-26820,FFS,NaN,134.33333,1615.36737,1571.66835,0.00000,1.588136e+06,0.00000,1.326359e+06,1.559343e+06,26.50984,14.50984,30.52918,138.10190,12.10190,159.59043,2.0,2.0,5.48214,40.0,NaN,157.26190,48.00000,NaN,61.11761,874.00000,NaN,1061.20297,104.00000,NaN,117.30828,10.94227,10.94227,24.13127,77.00000,NaN,147.56760,49.66007,5.66007,130.44540,11.0,NaN,17.26432,NaN,NaN,5.89768,47.02285,2.02285,47.95079,1.0,1.0,11.09491,38.00000,11.00000,52.59237,96.00000,NaN,152.58745,19.96174,1.96174,162.99213,3083.0,NaN,1397.89254,277.00000,NaN,1152.79314
3,2021,00004B7C4023,TIN,474454561.0,RINDFLEISCH FAMILY PRACTICE PLLC,9335446202,ID-26820,Humana Inc.,NaN,4.08333,64.41775,62.08302,62425.64956,6.159470e+04,56951.49748,6.149454e+04,4.990221e+04,1.00000,1.00000,0.90741,3.06541,3.06541,5.35777,NaN,NaN,0.21470,NaN,NaN,5.08571,1.14171,1.14171,2.79650,54.40250,17.40250,34.43227,1.00000,1.00000,3.60509,2.80669,2.80669,1.00841,8.79211,8.79211,5.05959,2.04853,2.04853,3.75260,NaN,NaN,0.48880,NaN,NaN,0.23227,2.95016,2.95016,2.01118,NaN,NaN,0.38286,3.86794,3.86794,2.26382,2.53870,2.53870,3.63422,41.00000,1.00000,5.26067,110.0,NaN,38.14688,35.05196,9.05196,46.10874
4,2021,00004B7C4023,TIN,474454561.0,RINDFLEISCH FAMILY PRACTICE PLLC,9335446202,ID-26820,ID Other Carriers,NaN,2.00000,6.49522,7.28228,6369.01941,6.367185e+03,5408.35426,9.439422e+03,7.609643e+03,NaN,NaN,0.09128,NaN,NaN,0.52406,NaN,NaN,0.00882,NaN,NaN,0.26644,NaN,NaN,0.21946,9.58539,9.58539,7.60774,NaN,NaN,0.53306,NaN,NaN,0.16346,NaN,NaN,0.82078,

In [6]:
conn = cb_utils.get_engine(source='msh_analytics')
table_name = f'{table_prefix}_data_tables_{year}_{current_folder}'
schema = 'raw'
print(f'select * from {schema}.{table_name}')
# pro_df.to_sql(pro_table_name, conn, schema=pro_schema, index=False, method='multi', chunksize=1000) # , if_exists='replace'

select * from raw.milliman_mani_data_tables_2021_20241203


In [7]:
mani_data_tables_2020_df.to_sql(table_name, conn, schema=schema, index=False, method='multi', chunksize=1000) # , if_exists='replace'

351611

In [8]:
file_name = 'MANI CY2022 Data Tables - v2024.2.0 - Nationwide.csv'
year = 2022
mani_data_tables_2021_df = pd.read_csv(f'{data_dir}/{current_folder}/{file_name}',  encoding='latin', low_memory=False)
# display(mani_data_tables_2020_df.head())
mani_data_tables_2021_df = cb_utils.df_format_columns(mani_data_tables_2021_df)
mani_data_tables_2021_df.head()

,ï»¿year,aco_builder_id,tin_ccn_type,tin_ccn_id,provider_name,pac_id,st_msa,carrier,member_status,py,risk_score_mmwtd,risk_score_mmwtd_region,ma_benchmark_rate_carrier,ma_benchmark_rate_0pct,ma_bid_revenue,paid_atffsreimb,paid_atffsreimb_region,ip_admits,ip_admits_imputed,ip_admits_region,ip_days,ip_days_imputed,ip_days_region,snf_admits,snf_admits_imputed,snf_admits_region,snf_days,snf_days_imputed,snf_days_region,ed,ed_imputed,ed_region,phy_office,phy_office_imputed,phy_office_region,rad_adv_imag_op,rad_adv_imag_op_imputed,rad_adv_imag_op_region,rad_adv_imag_nonhosp,rad_adv_imag_nonhosp_imputed,rad_adv_imag_nonhosp_region,rad_gen_op,rad_gen_op_imputed,rad_gen_op_region,rad_gen_nonhosp,rad_gen_nonhosp_imputed,rad_gen_nonhosp_region,rad_therap_op,rad_therap_op_imputed,rad_therap_op_region,rad_therap_nonhosp,rad_therap_nonhosp_imputed,rad_therap_nonhosp_region,prev_physicals,prev_physicals_imputed,prev_physicals_region,surgery_asc,surgery_asc_imputed,surgery_asc_region,surgery_op,surgery_op_imputed,surgery_op_region,part_b_rx_nonhosp,part_b_rx_nonhosp_imputed,part_b_rx_nonhosp_region,part_b_rx_op,part_b_rx_op_imputed,part_b_rx_op_region,lab_nonhosp,lab_nonhosp_imputed,lab_nonhosp_region,lab_op,lab_op_imputed,lab_op_region
0,2022,00004B7C4023,TIN,474454561.0,RINDFLEISCH FAMILY PRACTICE PLLC,9335446202,ID-26820,"Blue Cross of Idaho Health Service, Inc.",NaN,67.08333,688.71857,675.83032,714627.70490,7.145800e+05,581103.36340,4.962433e+05,6.234183e+05,3.52369,3.52369,9.56422,22.87855,22.87855,60.29091,1.41483,1.41483,2.08149,34.0,NaN,46.15165,16.43732,16.43732,32.52659,470.00000,NaN,473.74577,15.50679,15.50679,44.65937,6.85401,6.85401,9.49213,41.85973,2.85973,56.32547,46.88106,5.88106,68.14668,NaN,NaN,6.75570,19.0,NaN,4.47954,44.62204,7.62204,38.17067,8.22915,8.22915,6.60110,23.52649,4.52649,32.96683,43.10258,9.10258,62.20936,18.0,2.0,60.62409,1088.0,NaN,713.54484,152.50130,7.50130,436.03680
1,2022,00004B7C4023,TIN,474454561.0,RINDFLEISCH FAMILY PRACTICE PLLC,9335446202,ID-26820,FFS,NaN,121.66667,1343.30501,1298.78617,0.00000,1.397050e+06,0.00000,1.347886e+06,1.295010e+06,37.38080,20.38080,23.75523,295.01085,4.01085,128.45655,2.49247,2.49247,5.25985,68.0,NaN,129.31867,60.00000,NaN,55.77226,938.00000,NaN,903.83415,98.00000,NaN,82.54304,3.70142,3.70142,23.06506,86.00000,NaN,106.47740,70.00000,NaN,120.01398,3.73658,3.73658,15.21148,NaN,NaN,5.52193,36.73723,5.73723,47.19721,10.34100,10.34100,9.46332,30.33249,2.33249,43.68030,140.00000,NaN,130.08748,32.0,1.0,129.58221,2246.0,NaN,1219.14329,457.00000,NaN,1017.01127
2,2022,00004B7C4023,TIN,474454561.0,RINDFLEISCH FAMILY PRACTICE PLLC,9335446202,ID-26820,Humana Inc.,NaN,4.00000,57.05591,52.75940,59176.74676,5.897296e+04,51658.86983,9.237441e+04,5.115396e+04,1.39584,1.39584,0.89220,7.32997,7.32997,7.56576,NaN,NaN,0.15612,NaN,NaN,3.59328,7.48101,7.48101,3.14172,44.11247,16.11247,35.88708,6.07840,6.07840,3.06936,1.00000,1.00000,0.55152,3.71026,3.71026,4.48860,2.00000,2.00000,4.63020,NaN,NaN,0.13332,NaN,NaN,0.12924,2.00000,2.00000,1.88220,NaN,NaN,0.33540,5.26058,5.26058,2.22684,1.00000,1.00000,3.34200,24.0,1.0,5.75484,86.0,NaN,44.29764,14.08546,3.08546,39.08424
3,2022,00004B7C4023,TIN,474454561.0,RINDFLEISCH FAMILY PRACTICE PLLC,9335446202,ID-26820,ID Other Carriers,NaN,1.00000,10.06988,9.18409,10791.58654,1.013191e+04,7754.42628,3.162503e+03,6.777141e+03,NaN,NaN,0.11280,NaN,NaN,0.58632,NaN,NaN,0.02640,NaN,NaN,0.40956,NaN,NaN,0.27144,3.40616,3.40616,6.18324,NaN,NaN,0.56820,NaN,NaN,0.12684,NaN,NaN,0.71340,NaN,NaN,0.78336,NaN,NaN,0.06852,NaN,NaN,0.06168,1.00000,1.00000,0.82332,NaN,NaN,0.09540,NaN,NaN,0.27216,1.00000,1.00000,0.77220,NaN,NaN,0.41532,12.0,NaN,8.93112,NaN,NaN,3.84444
4,2022,00004B7C4023,TIN,474454561.0,RINDFLEISCH FAMILY PRACTICE PLLC,9335446202,ID-26820,"Molina Healthcare, Inc.,",NaN,5.33333,125.73039,85.99727,130678.95200,1.306790e+05,111278.94340,1.062688e+04,9.116816e+04,NaN,NaN,1.80332,NaN,NaN,13.79512,NaN,NaN,0.70236,NaN,NaN,16.55824,2.21767,2.21767,6.26464,47.43603,11.43603,4

In [9]:
conn = cb_utils.get_engine(source='msh_analytics')
table_name = f'{table_prefix}_data_tables_{year}_{current_folder}'
schema = 'raw'
print(f'select * from {schema}.{table_name}')
# pro_df.to_sql(pro_table_name, conn, schema=pro_schema, index=False, method='multi', chunksize=1000) # , if_exists='replace'

select * from raw.milliman_mani_data_tables_2022_20241203


In [10]:
mani_data_tables_2021_df.to_sql(table_name, conn, schema=schema, index=False, method='multi', chunksize=1000) # , if_exists='replace'

361014

# Next time

In [ ]:
data_table_yrs = [
    {yr, file_name, encoding}
]

In [ ]:
for {yr, file_name, encoding} in data_table_yrs:
    
    if encoding = None:
        encoding = 'latin'
        
    mani_data_tables_df = pd.read_csv(f'{data_dir}/{current_folder}/{file_name}',  encoding=encoding, low_memory=False)
    # display(mani_data_tables_2020_df.head())
    mani_data_tables_df = cb_utils.df_format_columns(mani_data_tables_df)
    conn = cb_utils.get_engine(source='msh_analytics')
    table_name = f'{table_prefix}_data_tables_{year}_{current_folder}'
    schema = 'raw'
    print(f'select * from {schema}.{table_name}')
    mani_data_tables_df.to_sql(table_name, conn, schema=schema, index=False, method='multi', chunksize=1000) # , if_exists='replace'